In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from google.colab import files
import nltk
import string
import re

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
dataset=pd.read_csv('/content/drive/MyDrive/BC_Test/Input.csv')[['URL_ID','URL']]

In [7]:
punc=[punc for punc in string.punctuation]
punc

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [9]:
#importing stop words files that are provided
StopWords_Auditor = pd.read_csv("/content/drive/MyDrive/BC_Test/StopWords/StopWords_Auditor.txt", header=None)[0].tolist()
StopWords_Currencies = pd.read_csv("/content/drive/MyDrive/BC_Test/StopWords/StopWords_Currencies.txt", header=None, encoding="ISO-8859-1", sep='delimiter')[0].tolist()
StopWords_DatesandNumbers = pd.read_csv("/content/drive/MyDrive/BC_Test/StopWords/StopWords_DatesandNumbers.txt", header=None)[0].tolist()
StopWords_Generic = pd.read_csv("/content/drive/MyDrive/BC_Test/StopWords/StopWords_Generic.txt", header=None)[0].tolist()
StopWords_GenericLong = pd.read_csv("/content/drive/MyDrive/BC_Test/StopWords/StopWords_GenericLong.txt", header=None)[0].tolist()
StopWords_Geographic = pd.read_csv("/content/drive/MyDrive/BC_Test/StopWords/StopWords_Geographic.txt", header=None)[0].tolist()
StopWords_Names = pd.read_csv("/content/drive/MyDrive/BC_Test/StopWords/StopWords_Names.txt", header=None)[0].tolist()

<ipython-input-9-bbbe7b8418d7>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  StopWords_Currencies = pd.read_csv("/content/drive/MyDrive/BC_Test/StopWords/StopWords_Currencies.txt", header=None, encoding="ISO-8859-1", sep='delimiter')[0].tolist()


In [10]:
#creating func for removing stop words
def text_process(text):
    nopunc =[char for char in text if char not in punc or char not in [':',',','(',')','’','?']]
    nopunc=''.join(nopunc)
    txt=' '.join([word for word in nopunc.split() if word.lower() not in StopWords_Auditor])
    txt1=' '.join([word for word in txt.split() if word.lower() not in StopWords_Currencies])
    txt2=' '.join([word for word in txt1.split() if word.lower() not in StopWords_DatesandNumbers])
    txt3=' '.join([word for word in txt2.split() if word.lower() not in StopWords_Generic])
    txt4=' '.join([word for word in txt3.split() if word.lower() not in StopWords_GenericLong])
    txt5=' '.join([word for word in txt4.split() if word.lower() not in StopWords_Geographic])
    return ' '.join([word for word in txt5.split() if word.lower() not in StopWords_Names])


In [11]:
#importing master Dictionary
positive=pd.read_csv("/content/drive/MyDrive/BC_Test/MasterDictionary/positive-words.txt", names=["Positive text"],header=None)
negative=pd.read_csv("/content/drive/MyDrive/BC_Test/MasterDictionary/negative-words.txt", names=["Negative text"],header=None,encoding="ISO-8859-1",sep='delimiter' )

<ipython-input-11-40dff0f77a5e>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  negative=pd.read_csv("/content/drive/MyDrive/BC_Test/MasterDictionary/negative-words.txt", names=["Negative text"],header=None,encoding="ISO-8859-1",sep='delimiter' )


In [12]:
#positive and negative dictionary without stopwords
positive['Positive text']=positive['Positive text'].apply(text_process)
negative['Negative text']=negative['Negative text'].apply(text_process)

In [13]:
# Creating final positive and negative list
positiveWordList = positive['Positive text'].tolist()
negativeWordList = negative['Negative text'].tolist()

In [14]:
output = pd.DataFrame(columns=['URL_ID', 'URL', 'Positive Score', 'Negative Score', 'Polarity Score', 'Subjectivity Score', 'Average Sentence Length', 'Percentage of Complex Words','Fog Index', 'Average Number of Words Per Sentence', 'Complex Word Count', 'Word Count', 'Syllables per Word', 'Personal Pronouns', 'Average Word Length'])

In [15]:

# Iterate through each URL in the dataframe
for i, row in dataset.iterrows():
    url_id = row['URL_ID']
    url = row['URL']

    try:
        print(f"Processing article {i + 1}: {url_id} - {url}")
        # Fetch HTML content
        page = requests.get(url, headers={"User-Agent": "XY"})
        soup = BeautifulSoup(page.text, 'html.parser')

        # Extract article text
        text = soup.find(attrs={'class': 'td-post-content'}).get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = ' '.join(chunk for chunk in chunks if chunk)

        # Tokenization
        tokenize_text = nltk.word_tokenize(text)

        # Positive score
        positive_score = sum(1 for word in tokenize_text if word.lower() in positiveWordList)

        # Negative score
        negative_score = sum(1 for word in tokenize_text if word.lower() in negativeWordList)

        # Polarity score
        x = positive_score - negative_score
        y = positive_score + negative_score
        polarity_score = (x / y) + 0.000001

        # Subjectivity score
        subjectivity_score = y / (len(tokenize_text) + 0.000001)

        # Average sentence length
        num_sentences = len(nltk.sent_tokenize(text))
        words_ = sum(len(sentence) for sentence in nltk.sent_tokenize(text))
        avg_sentence_length = words_ / num_sentences

        # Percentage of complex words
        def is_complex_word(word):
            d = {}.fromkeys('aeiou', 0)
            has_lots_of_vowels = False

            for x in word.lower():
                if x in d:
                    d[x] += 1

            for q in d.values():
                if q > 2:
                    has_lots_of_vowels = True

            if word.lower() not in string.punctuation and has_lots_of_vowels:
                return True
            return False

        complex_words = sum(1 for word in tokenize_text if is_complex_word(word))
        percentage_of_complex_words = complex_words / len(tokenize_text)

        # Fog Index
        fog_index = 0.4 * (avg_sentence_length + percentage_of_complex_words)

        # Average number of words per sentence
        avg_words_per_sentence = words_ / num_sentences

        # Complex word count
        complex_word_count = complex_words

        # Word count
        word_count = len(tokenize_text)

        # Syllables per word
        # Function to count syllables in a word
        def count_syllables(word):
            word = word.lower()
            if word in ['a', 'i', 'o', 'u', 'e']:
                return 1
            count = 0
            vowels = 'aeiou'
            if word[0] in vowels:
                count += 1
            for index in range(1, len(word)):
                if word[index] in vowels and word[index - 1] not in vowels:
                    count += 1
            if word.endswith('e'):
                count -= 1
            if word.endswith('le'):
                count += 1
            if count == 0:
                count += 1
            if word.endswith(('es', 'ed')):
                count -= 1
            return count

        syllable_count = sum(count_syllables(word) for word in tokenize_text)
        syllables_per_word = syllable_count / len(tokenize_text)

        # Personal pronouns count
        pronouns = ['i', 'we', 'my', 'ours', 'us']
        personal_pronouns_count = sum(1 for word in tokenize_text if word.lower() in pronouns)

        # Average word length
        total_chars = sum(len(word) for word in tokenize_text)
        average_word_length = total_chars / len(tokenize_text)

        # Append to output dataframe
        output = output.append({'URL_ID': url_id,
                                'URL': url,
                                'Positive Score': positive_score,
                                'Negative Score': negative_score,
                                'Polarity Score': polarity_score,
                                'Subjectivity Score': subjectivity_score,
                                'Average Sentence Length': avg_sentence_length,
                                'Percentage of Complex Words': percentage_of_complex_words,
                                'Fog Index': fog_index,
                                'Average Number of Words Per Sentence': avg_words_per_sentence,
                                'Complex Word Count': complex_word_count,
                                'Word Count': word_count,
                                'Syllables per Word': syllables_per_word,
                                'Personal Pronouns': personal_pronouns_count,
                                'Average Word Length': average_word_length}, ignore_index=True)

        print(f'Successfully processed article {i + 1}: {url}')

    except Exception as e:
        print(f'Error occurred while processing article {i + 1}: {e}')

Processing article 1: blackassign0001 - https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/


<ipython-input-15-ed0d1a691144>:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({'URL_ID': url_id,


Successfully processed article 1: https://insights.blackcoffer.com/rising-it-cities-and-its-impact-on-the-economy-environment-infrastructure-and-city-life-by-the-year-2040-2/
Processing article 2: blackassign0002 - https://insights.blackcoffer.com/rising-it-cities-and-their-impact-on-the-economy-environment-infrastructure-and-city-life-in-future/


<ipython-input-15-ed0d1a691144>:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({'URL_ID': url_id,


Successfully processed article 2: https://insights.blackcoffer.com/rising-it-cities-and-their-impact-on-the-economy-environment-infrastructure-and-city-life-in-future/
Processing article 3: blackassign0003 - https://insights.blackcoffer.com/internet-demands-evolution-communication-impact-and-2035s-alternative-pathways/


...

Successfully processed article 98: https://insights.blackcoffer.com/contribution-of-handicrafts-visual-arts-literature-in-the-indian-economy/
Processing article 99: blackassign0099 - https://insights.blackcoffer.com/how-covid-19-is-impacting-payment-preferences/


<ipython-input-15-ed0d1a691144>:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({'URL_ID': url_id,


Successfully processed article 99: https://insights.blackcoffer.com/how-covid-19-is-impacting-payment-preferences/
Processing article 100: blackassign0100 - https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-2/


<ipython-input-15-ed0d1a691144>:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append({'URL_ID': url_id,


Successfully processed article 100: https://insights.blackcoffer.com/how-will-covid-19-affect-the-world-of-work-2/
Processing article 101: nan - nan
Error occurred while processing article 101: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Processing article 102: nan - nan
Error occurred while processing article 102: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Processing article 103: nan - nan
Error occurred while processing article 103: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Processing article 104: nan - nan
Error occurred while processing article 104: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Processing article 105: nan - nan
Error occurred while processing article 105: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Processing article 106: nan - nan
Error occurred while processing article 106: Invalid URL 'nan': No scheme supplied. Perhaps you meant http

In [17]:
# Save the output dataframe to a file
output.to_excel('/content/drive/MyDrive/BC_Test/Answer.xlsx', index=False)

In [18]:
Output = pd.read_excel('/content/drive/MyDrive/BC_Test/Answer.xlsx')
Output

,URL_ID,URL,Positive Score,Negative Score,Polarity Score,Subjectivity Score,Average Sentence Length,Percentage of Complex Words,Fog Index,Average Number of Words Per Sentence,Complex Word Count,Word Count,Syllables per Word,Personal Pronouns,Average Word Length
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,33,6,0.692309,0.028446,88.371795,0.016047,35.355137,88.371795,22,1371,1.393144,11,4.193290
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,60,31,0.318682,0.053878,120.175000,0.031380,48.082552,120.175000,53,1689,1.633511,6,4.874482
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,38,24,0.225807,0.050903,135.052632,0.045977,54.039443,135.052632,56,1218,1.835796,13,5.491790
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,38,75,-0.327433,0.092699,143.557692,0.046760,57.441781,143.557692,57,1219,1.751436,5,5.301887
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,22,8,0.466668,0.039370,115.717949,0.043307,46.304502,115.717949,33,762,1.623360,6,5.080052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,blackassign0096,https://insights.blackcoffer.com/what-is-the-r...,28,57,-0.341175,0.069959,141.420000,0.028807,56.579523,141.420000,35,1215,1.608230,2,4.940741
94,blackassign0097,https://insights.blackcoffer.com/impact-of-cov...,22,35,-0.228069,0.047264,164.210526,0.027363,65.695156,164.210526,33,1206,1.401327,6,4.312604
95,blackassign0098,https://insights.blackcoffer.com/contribution-...,5,3,0.250001,0.018561,214.833333,0.027842,85.944470,214.833333,12,431,1.661253,0,5.118329
96,blackassign0099,https://insights.blackcoffer.com/how-covid-19-...,16,3,0.684212,0.027616,114.451613,0.010174,45.784715,114.451613,7,688,1.434593,4,4.312500
